In [1]:
from utils.compression import compress_directories, decompress_directories

In [ ]:
# base_path = "data/gwilliams"
# destination_path = "compressed_data/gwilliams"

# compress_directories(
#     base_path,
#     destination_path,
#     checksum_file_name="checksums.txt",
#     compression_level=12,
#     num_workers='mac'
# )

Compressing directories: 100%|██████████| 28/28 [14:06<00:00, 30.22s/it]

Copied .DS_Store to compressed_data/gwilliams
Copied dataset_description.json to compressed_data/gwilliams
Copied README.txt to compressed_data/gwilliams
Copied participants.json to compressed_data/gwilliams
Copied participants.tsv to compressed_data/gwilliams
Compression complete.


In [ ]:
# base_path = "data/schoffelen"
# destination_path = "compressed_data/schoffelen"

# compress_directories(
#     base_path,
#     destination_path,
#     checksum_file_name="checksums.txt",
#     compression_level=12,
#     num_workers='mac'
# )

In [ ]:
# base_path = "downloaded_data/gwilliams"
# destination_path = "data/gwilliams"

# decompress_directories(
#     base_path,
#     destination_path,
#     checksum_file_name="checksums.txt",
#     delete_compressed_files=True,
#     num_workers=None
# )

Decompressing 28 files using 12 processes...


Decompressing files:   0%|          | 0/28 [00:00<?, ?it/s]

Decompressing files: 100%|██████████| 28/28 [04:42<00:00, 10.09s/it]

Copied .DS_Store to data/gwilliams
Copied participants.json to data/gwilliams
Copied dataset_description.json to data/gwilliams
Copied README.txt to data/gwilliams
Copied participants.tsv to data/gwilliams
Decompression and verification complete
All files decompressed successfully.


In [ ]:
# base_path = "downloaded_data/schoffelen"
# destination_path = "data/schoffelen"

# decompress_directories(
#     base_path,
#     destination_path,
#     checksum_file_name="checksums.txt",
#     delete_compressed_files=True,
#     num_workers=12
# )

Decompressing 7 files using 12 processes...


Decompressing files: 100%|██████████| 7/7 [07:40<00:00, 65.74s/it] 

Copied .DS_Store to data/schoffelen
Copied LICENSE.txt to data/schoffelen
Copied dataset_description.json to data/schoffelen
Copied MANIFEST.txt to data/schoffelen
Copied README.txt to data/schoffelen
Copied participants.tsv to data/schoffelen
Decompression and verification complete
All files decompressed successfully.


In [1]:
# data_partition = {
#         # "gwilliams2023": {
#         #     "testing_subjects": [19, 20, 21],
#         #     "testing_tasks": [0],
#         # }
#         "schoffelen2022": {
#             "testing_subjects": [],
#             "testing_tasks": [8, 9],
#         },
# }

# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=3
#     )

Decompressing 7 files using 3 processes...


Decompressing files: 100%|██████████| 7/7 [06:51<00:00, 58.81s/it] 


Copied dataset_description.json to data/schoffelen2022
Copied README.txt to data/schoffelen2022
Copied participants.tsv to data/schoffelen2022
Copied MANIFEST.txt to data/schoffelen2022
Copied .DS_Store to data/schoffelen2022
Copied LICENSE.txt to data/schoffelen2022
Decompression and verification complete
All files decompressed successfully.
Deleting compressed files...
